In [24]:
import io
import requests
import numpy as np
import pandas as pd
from collections import namedtuple

# Datasets

## Government Measurement Dataset

In [25]:
# Oxford Covid-19 Government Response Tracker (OxCGRT)
oxcgrt_url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
oxcgrt_data = requests.get(oxcgrt_url).content
oxcgrt_data = pd.read_csv(io.StringIO(oxcgrt_data.decode('utf-8')))

## Testing Cases Dataset

In [26]:
test_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv'
test_data = requests.get(test_url).content
test_data = pd.read_csv(io.StringIO(test_data.decode('utf-8')))

## Confirmed Cases Dataset

In [27]:
confirmed_url = 'https://datahub.io/core/covid-19/r/time-series-19-covid-combined.csv'
confirmed_data = requests.get(confirmed_url).content
confirmed_data = pd.read_csv(io.StringIO(confirmed_data.decode('utf-8')))

# Data Cleaning

#### Due to different date format adopted by different dataset, we need to unify date format across different datasets.

In [28]:
oxcgrt_data['Date'] = oxcgrt_data['Date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
test_data['Date'] = test_data['Date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
confirmed_data['Date'] = confirmed_data['Date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

#### Map from measurement name to its abbreviation and vice versa for government measure dataset.

In [32]:
a2m = {'C1': 'C1_School closing', 'C2': 'C2_Workplace closing', 'C3': 'C3_Cancel public events', 'C4': 'C4_Restrictions on gatherings', 
       'C5': 'C5_Close public transport', 'C6': 'C6_Stay at home requirements', 'C7': 'C7_Restrictions on internal movement', 
       'C8': 'C8_International travel controls', 'H1': 'H1_Public information campaigns'}
m2a = {v: k for k, v in a2m.items()}

#### Government measuers is divided into 18 indicators (C1 ~ C8, E1 ~ E4, H1 ~ H5, and M1), and we focus on C1 to C8 and H1,
#### because E1 ~ E4 are fiscal and monetary policies and H2 ~ H5 are testing and contact tracing policies and investment in health system.
#### A "Stringency Index" is also provided to measure the strigency of government measures.
#### More descriptions can be found: https://www.bsg.ox.ac.uk/sites/default/files/2020-05/BSG-WP-2020-032-v5.0_0.pdf,
#### and https://www.bsg.ox.ac.uk/sites/default/files/Calculation%20and%20presentation%20of%20the%20Stringency%20Index.pdf
#### Note that this dataset has been modified officially on April 30th with note: https://www.bsg.ox.ac.uk/sites/default/files/OxCGRT.%20What%27s%20changed%2024%20April%202020.pdf

In [34]:
# select C1 to C8, H1 and Stringency Index
dc_measure_data = dict()
Measure = namedtuple('Measure', 'C1 C2, C3, C4, C5, C6, C7, C8, H1, stringency')
for index, row in oxcgrt_data.iterrows():
    value = []
    for abbr, measure in a2m.items():
        if np.isnan(row[measure]):
            value.append(None)
        else:
            value.append(row[measure])
    if np.isnan(row['StringencyIndexForDisplay']):
        value.append(None)
    else:
        value.append(row['StringencyIndexForDisplay'])

    key = (row['Date'], row['CountryName'])
    value = Measure(*value)
    dc_measure_data[key] = value

#### Due to incomplete reports by different governments, there are some invalid (NAN) values in the govrenment measurement dataset.
#### We refill invalid values by the following steps:
1. sort dictionary by keys which are tuples: first by country, second by date
2. refill None values using the value of the before/after days, as measures won't change significantly
3. delete a data sample if we cannot find a valid value in maximum 7 before/after days

In [35]:
dc_measure_data = {elem[0]: elem[1] for elem in sorted(dc_measure_data.items(), key=lambda x: (x[0][1], x[0][0]))}
delete_key = []
for key, value in dc_measure_data.items():
    for s, v in value._asdict().items():
        if v is None:
            refill = False
            # forward pass
            cnt = 1
            prev_date = key[0] + pd.DateOffset(-1)
            prev_key = (prev_date, key[1])
            while cnt < 7:
                if dc_measure_data.get(prev_key, None) is not None:
                    prev_v = getattr(dc_measure_data.get(prev_key, None), s)
                    if prev_v is not None:
                        dc_measure_data[key] = dc_measure_data[key]._replace(**{s: prev_v})
                        refill = True
                        break
                    else:
                        prev_date = prev_date + pd.DateOffset(-1)
                        prev_key = (prev_date, key[1])
                        cnt += 1
                else:
                    prev_date = prev_date + pd.DateOffset(-1)
                    prev_key = (prev_date, key[1])
                    cnt += 1
            if not refill:
                cnt = 1
                after_date = key[0] + pd.DateOffset(1)
                after_key = (after_date, key[1])
                while cnt < 7:
                    if dc_measure_data.get(after_key, None) is not None:
                        after_v = getattr(dc_measure_data.get(after_key, None), s)
                        if after_v is not None:
                            dc_measure_data[key] = dc_measure_data[key]._replace(**{s: after_v})
                            refill = True
                            break
                        else:
                            after_date = after_date + pd.DateOffset(-1)
                            after_key = (after_date, key[1])
                            cnt += 1
                    else:
                        after_date = after_date + pd.DateOffset(-1)
                        after_key = (after_date, key[1])
                        cnt += 1
                if not refill:
                    delete_key.append(key)
                    break

dc_measure_data = {k: v for k, v in dc_measure_data.items() if k not in delete_key}

#### In the test cases dataset, different countries report test numbers under one or more standards, including:
#### 'tests performed', 'cases tested', 'people tested', ..., and 'unit unclear'.
#### More information can be found: https://ourworldindata.org/covid-testing#our-checklist-for-covid-19-testing-data.
#### To keep consistency, we only keep the test numbers under one standard which has the maximum available data samples.
#### Note that as long as the standard in any single country is the same along the timeline, the analysis is valid.
#### Steps are as follows:

Step 1: get the number of available data for each country under each standard

In [36]:
sc_tested_dict = dict()     # 'sc' means standard and country
for index, row in test_data.iterrows():
    entity = row['Entity']
    country, standard = entity.split('-')
    country = country.strip()
    standard = standard.strip()
    if country not in sc_tested_dict.keys():
        sc_tested_dict[country] = {standard: 1}
    else:
        if standard not in sc_tested_dict[country]:
            sc_tested_dict[country][standard] = 1
        else:
            sc_tested_dict[country][standard] += 1

Step 2: find the standard with the maximum number of availabel data for each country

In [37]:
ssc_tested_dict = dict()    # 'ssc' means single standard and country
for country, value in sc_tested_dict.items():
    sorted_value = {k: v for k, v in sorted(value.items(), key=lambda item: -item[1])}
    standard = list(sorted_value)[0]
    ssc_tested_dict[country] = standard

Step 3: a). remove data that is not consistent with the selected standard; b). reformat Entity name

In [38]:
for index, row in test_data.iterrows():
    entity = row['Entity']
    country, standard = entity.split('-')
    country = country.strip()
    standard = standard.strip()
    if standard != ssc_tested_dict[country]:
        test_data.drop(index, inplace=True)
    else:
        test_data.loc[index, 'Entity'] = country

#### Record the number of daily test cases using the number of cumulative cases.

In [39]:
dc_tested_dict = dict()
Tested = namedtuple('Tested', 'cumulative daily')
for index, row in test_data.iterrows():
    key = (row['Date'], row['Entity'])
    if not np.isnan(row['Daily change in cumulative total']):
        dc_tested_dict[key] = Tested(int(row['Cumulative total']), int(row['Daily change in cumulative total']))
    else:
        dc_tested_dict[key] = Tested(int(row['Cumulative total']), None)

#### In the confirmed cases dataset, the number of cummulative confirmed cases in some countries are inconsistent, and we need to:
#### 1. Remove invalid (NaN) number of confirmed cases.
#### 2. For some countries, combine all province/state statistics into a single country-level statistic.

In [40]:
dc_confirmed_dict = dict()  # 'dc' means date and country
for index, row in confirmed_data.iterrows():
    key = (row['Date'], row['Country/Region'])
    if not np.isnan(row['Confirmed']):
        if key in dc_confirmed_dict.keys():
            dc_confirmed_dict[key] = dc_confirmed_dict[key] + int(row['Confirmed'])
        else:
            dc_confirmed_dict[key] = int(row['Confirmed'])

#### compute daily confirmed cases using the number of cumulative cases.

In [41]:
Confirmed = namedtuple('Confirmed', 'cumulative daily')
for key, value in dc_confirmed_dict.items():
    prev_date_key = (key[0] + pd.DateOffset(-1), key[1])
    if prev_date_key in dc_confirmed_dict.keys():
        if not isinstance(dc_confirmed_dict[prev_date_key], Confirmed):
            dc_confirmed_dict[key] = Confirmed(value, value - dc_confirmed_dict[prev_date_key])
        else:
            dc_confirmed_dict[key] = Confirmed(value, value - dc_confirmed_dict[prev_date_key].cumulative)
    else:
        dc_confirmed_dict[key] = Confirmed(value, None)

# Date Integration

#### Combine data from three datasets together using dictionary structure: key - (date, country), value - (Measure, Confirmed, Tested).
#### Because the three datasets are collected by different organizations/groups, we need to filter out incomplete data samples.
#### Each item represent a sample for a country in a day which appears in all three datasets.

In [42]:
dc_combined_data = dict()   # 'dc' means date and country
Combined = namedtuple('Combined', 'measure confirmed tested')
for key, value in dc_measure_data.items():
    if key in dc_confirmed_dict and key in dc_tested_dict:
        dc_combined_data[key] = Combined(value, dc_confirmed_dict[key], dc_tested_dict[key])

# Have a look at the final combined dataset

In [43]:
print('Total number of samples: {}'.format(len(dc_combined_data)))

Total number of samples: 2822


In [48]:
import random

key = random.choice(list(dc_combined_data.keys()))
print('A random sample in the integrated dataset:')
print('key: ', key)
print('value: ', dc_combined_data[key])

A random sample in the integrated dataset:
key:  (Timestamp('2020-04-24 00:00:00'), 'Turkey')
value:  Combined(measure=Measure(C1=3.0, C2=2.0, C3=2.0, C4=0.0, C5=2.0, C6=2.0, C7=2.0, C8=4.0, H1=2.0, stringency=80.42), confirmed=Confirmed(cumulative=104912, daily=3122), tested=Tested(cumulative=830257, daily=38351))


### Save the cleaned and integrated data into a file

In [51]:
import pickle

filename = 'cleaned_integrated_data.pkl'

f = open(filename, 'wb')
pickle.dump(dc_combined_data, f)
f.close()

### Code to load the cleaned and integrated data

In [53]:
with open(filename, 'rb') as fp:
    loaded = pickle.load(fp)
    
print('Total number of samples: {}'.format(len(dc_combined_data)))

key = random.choice(list(dc_combined_data.keys()))
print('A random sample in the integrated dataset:')
print('key: ', key)
print('value: ', dc_combined_data[key])

Total number of samples: 2822
A random sample in the integrated dataset:
key:  (Timestamp('2020-03-23 00:00:00'), 'Zimbabwe')
value:  Combined(measure=Measure(C1=0.0, C2=0.0, C3=2.0, C4=3.0, C5=0.0, C6=1.0, C7=1.0, C8=4.0, H1=1.0, stringency=51.45), confirmed=Confirmed(cumulative=3, daily=0), tested=Tested(cumulative=15, daily=None))
